#### Instala os módulos necessários

In [ ]:
%pip install duckdb
%pip install keras
%pip install nbformat
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install pydot
%pip install scikit-learn
%pip install seaborn
%pip install tensorflow
%pip install warnings

#### Importa os módulos necessários

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, SimpleRNN
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from tensorflow.keras.utils import plot_model
import duckdb as ddb
import nbformat
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import warnings

#### Ignora warnings que serão gerados

In [ ]:
warnings.filterwarnings("ignore")

#### Análise exploratória, feature engineering, treinamento e avaliação do modelo

In [ ]:
raw_bbas3 = pd.read_csv("_datasets/bbas3/treino.csv", usecols=range(2,20))

Ao importar o conjunto de dados, notamos que a primeira coluna equivalia ao índice da linha, portanto, utilizamos o parâmetro "usecols" do método "read_csv" para removê-la, podendo assim, ler apenas os dados de treinamento necessários para construção do classificador.

In [ ]:
raw_bbas3.head(5)

In [ ]:
raw_bbas3.info()

Conforme apresentado acima, notamos que nenhuma das variáveis apresenta valores nulos, portanto, neste momento, não precisaremos remover ou realizar a inputação de valores como tratamento a valores ausentes. 

Dessa forma, como próximos passos, iremos analisar o comportamento das variáveis, a fim de entender se o conjunto de dados é balanceado (número de compras e vendas similar) e, em seguida, verificar a correlação entre as variáveis.

In [ ]:
label_distribution = ddb.sql("""SELECT Label, COUNT(*) AS Regs from raw_bbas3 GROUP BY 1""").to_df()
size = raw_bbas3.shape[0]
buy_count = label_distribution["Regs"][0]
sell_count = label_distribution["Regs"][1]
buy_percentual = round((buy_count / size) * 100, 2)
sell_percentual = round((sell_count / size) * 100, 2)

print(f"O conjunto de dados possui {size} registros, onde:")
print(f" -> {buy_count} ou {buy_percentual}% são vendas")
print(f" -> {sell_count} ou {sell_percentual}% são compras")

O conjunto de dados apresenta uma boa distribuição entre as classes, sendo assim, não será necessário aplicar nenhuma técnica de amostragem para balancear os dados, o geralmente é feito para evitar que o modelo tornesse especialista em uma única classe.

In [ ]:
correlation = raw_bbas3.select_dtypes("number").corr()

plt.figure(figsize=(12, 4))

sns.heatmap(
    correlation, 
    annot=True, 
    annot_kws={"size" : 10},
    cmap="coolwarm",
    cbar=True,
    linewidths=0.5,
    linecolor="white"
)

plt.title("\nCorrelação entre as variáveis\n", fontdict={"size" : 12})

plt.show()

Nesse momento, há dois comportamentos importantes a serem notados:

- Variável dependente: a correlação entre a ação de venda/compra diminui ao passar dos dias, ou seja, considerando APENAS o que é apresentado acima, poderíamos pressupor que a ação é tomada com base nos dados mais recentes, ou seja, uma janela mais curta de dias.

- Variáveis independentes: diferente da variável dependente, todas as variáveis independentes apresentam uma alta correlação entre sí, o que nos leva a pensar que o valor de fechamento em D-15 influencia em D-14 e assim sucessivamente.

Dito isso, como bons cientistas de dados, começaremos a realizar os experimentos!